In [3]:
import pandas as pd

In [5]:
df = pd.read_csv('sample_train.csv')

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from transformers import TFDistilBertModel, DistilBertTokenizer

2025-01-30 12:49:11.525302: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-30 12:49:12.528094: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738259352.723756   13575 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738259352.779113   13575 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-30 12:49:13.235095: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
bert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

I0000 00:00:1738259367.250611   13575 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTo

In [6]:
def tokenize_data(texts, tokenizer, max_length):
    return tokenizer(
        texts, 
        max_length=max_length, 
        padding='max_length', 
        truncation=True, 
        return_tensors='tf'
    )

texts = df['text'].tolist()
labels = df['status'].tolist()
max_length = 128
tokenized_data = tokenize_data(texts, tokenizer, max_length)


In [9]:
class CustomModel(tf.keras.Model):
    def __init__(self, bert_model):
        super(CustomModel, self).__init__()
        self.bert = bert_model
        self.lstm = LSTM(128, return_sequences=True)
        self.dropout = Dropout(0.3)
        self.dense = Dense(1, activation='sigmoid')

    def call(self, inputs):
        bert_output = self.bert(inputs)[0]
        lstm_output = self.lstm(bert_output)
        dropout_output = self.dropout(lstm_output)
        output = self.dense(dropout_output[:, -1, :])
        return output

model = CustomModel(bert_model)


In [12]:
# Create a tf.data.Dataset from the tokenized data and labels
dataset = tf.data.Dataset.from_tensor_slices((
    {'input_ids': tokenized_data['input_ids'], 'attention_mask': tokenized_data['attention_mask']},
    labels
)).batch(8)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(
    dataset,
    epochs=3
)


Epoch 1/3


I0000 00:00:1738259638.496040   13725 cuda_dnn.cc:529] Loaded cuDNN version 90300


8/8 ━━━━━━━━━━━━━━━━━━━━ 13s 98ms/step - accuracy: 0.6527 - loss: 1.0085
Epoch 2/3
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - accuracy: 0.9038 - loss: 0.2584
Epoch 3/3
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - accuracy: 0.7204 - loss: 0.5595


In [16]:
pred_data = ["A senior BlackRock Inc executive made donation unsuccessful U.S. presidential candidate last year, action may prohibit world’s largest asset manager collecting fees state government, company regulatory filing showed. Mark Wiedman, BlackRock senior managing director, donated $2,700 presidential campaign Ohio Governor John Kasich, seeking Republican Party nomination, according filing made earlier year U.S. Securities Exchange Commission reviewed Reuters Monday. Federal securities rules prohibit companies executive officers making donations government officials could influence hiring fund manager authority appoint person could providing asset management services governments fee. The ban effect two years contribution made. The state Ohio uses BlackRock-managed funds, relationship BlackRock “substantially predates” donation, filing said. In May filing, BlackRock asked SEC permission accept fees state Ohio, saying exceptions rule granted before. Without ruling, BlackRock said could face loss approximately $37 million. BlackRock said Wiedman asked received refund campaign contribution Kasich, “made personal political beliefs contributor desire influence award investment advisory business.” The SEC yet responded BlackRock’s request. In emailed statement, BlackRock said application “fair reasonable, consistent intent relevant rule.” “The contribution question made solely support Mr. Kasich’s presidential campaign purpose,” BlackRock spokeswoman Tara McDonnell said statement. “Through BlackRock’s robust policies procedures, discovered contribution question, BlackRock Mr. Wiedman worked together promptly address error, including obtaining refund $2,700 contribution,” McDonnell said. A spokesman SEC, Ryan White, declined comment filing. A spokesman Kasich immediately respond request comment donation. Wiedman global head iShares index investments, booming business within BlackRock includes exchange-traded funds. The iShares brand brought nearly $74 billion new cash BlackRock second quarter. BlackRock manages $5.7 trillion overall."]
tokenized_pred_data = tokenize_data(pred_data, tokenizer, max_length)

In [17]:
model.predict({'input_ids': tokenized_pred_data['input_ids'], 'attention_mask': tokenized_pred_data['attention_mask']})

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step


array([[0.3667906]], dtype=float32)

In [19]:
loss, accuracy = model.evaluate(dataset)
print(f"Loss: {loss}, Accuracy: {accuracy}")

8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 67ms/step - accuracy: 0.6809 - loss: 0.5652
Loss: 0.45474356412887573, Accuracy: 0.7903226017951965
